In [3]:
# il faudra installer selenium si c'est pas encore fait
#!pip install selenium

In [4]:
from bs4 import BeautifulSoup 
from selenium import webdriver
# The following packages will also be used in this tutorial
import pandas as pd # All database operations
import numpy as np  # Numerical operations
import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation
from sklearn.feature_extraction.text import CountVectorizer # BagOfWords (cleaning)
from joblib import Parallel, delayed # Parallellization of tasks

In [25]:
def getPage(url):
	''' returns a soup object that contains all the information 
	of a certain webpage'''
	result = requests.get(url)
	content = result.content
	return BeautifulSoup(content, features = "lxml")

url_page ="https://www.airbnb.fr/s/Paris/homes?adults=1&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&checkin=2020-11-25&checkout=2020-11-28&source=structured_search_input_header&search_type=filter_change"
page=getPage(url_page)

In [26]:
def getRoomClasses(soupPage):
	''' This function returns all the listings that can 
	be found on the page in a list.'''
	rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
	result = []
	for room in rooms:
		result.append(room)
	return result
listing=getRoomClasses(page)

In [27]:
##commentaires
def getNbCommentaire(listing):
	''' Returns the guest information'''
	try:
		return listing.find("span", {"class":"_a7a5sx"}).text 
	except:
		return "pas de commentaire"

In [28]:
##evaluation
def getEvaluation(listing):
	''' Returns the guest information'''
	try:
		return listing.find("span", {"class":"_10fy1f8"}).text 
	except:
		return "pas d'évaluation"


In [29]:
## lien du logement
def getListingLink(listing):
	''' Returns the guest information'''
	try:
		return "http://airbnb.com" + listing.find("a")["href"]
	except:
		return "pas de lien"

In [30]:
## titre du logement
def getListingTitle(listing):
	''' Returns the guest information'''
	try:
		return listing.find("meta")["content"]
	except:
		return "pas de titre du logement"

In [31]:
##description du logement
def getTopRow(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class": "_167qordg"}).text
	except:
		return "pas de description du logement"

In [32]:
##information sur le nombre de chambre/douche
def getRoomInfo(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class":"_kqh46o"}).text
	except:
		return "pas d'infos sur chambre/douche"

In [33]:
##prix
def getPrix(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class":"_1fwiw8gv"}).text
	except:
		return "pas d'infos sur le prix"

In [34]:

getNbCommentaire(listing[5])
getPrix(listing[0])
getEvaluation(listing[5])
getListingTitle(listing[6])

'Grand F1 au cœur de Paris / Montorgueil - null - Paris'

In [35]:
## extraction des infos de tous les logements d'une page

def extractinfo(page):
    df = pd.DataFrame(columns = ['title', 'link',"nbComments","prix","topRow","evaluation","roomInfo"]) ##,"evaluation", ,'nbComments'
    new=[]
    listing=getRoomClasses(page)
    for i in range(1, len(listing)):
        new.append(getListingTitle(listing[i]))
        new.append(getListingLink(listing[i]))
        new.append(getNbCommentaire(listing[i]))
        new.append(getPrix(listing[i]))
        new.append(getTopRow(listing[i]))
        new.append(getEvaluation(listing[i]))
        new.append(getRoomInfo(listing[i]))
        df.loc[i]=new
        new=[]
    return df

In [36]:
extractinfo(page)

,title,link,nbComments,prix,topRow,evaluation,roomInfo
1,MAISON LE BAC Apt. 402B - null - Paris,http://airbnb.com/rooms/45727652?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 10e arrondissement,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
2,Appartement central - Louvre - null - Paris,http://airbnb.com/rooms/42825643?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 2e arrondissement,pas d'évaluation,4 voyageurs · 1 chambre · 2 lits · 1 salle de ...
3,Havre de paix Faubourg St Denis en bail mobili...,http://airbnb.com/rooms/18506150?adults=1&prev...,(22),pas d'infos sur le prix,Logement entier à 10e arrondissement,4.82,2 voyageurs · Studio · 1 lit · 1 salle de bain
4,studio quartier canal st Martin. (bail mobilit...,http://airbnb.com/rooms/45629307?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 10e arrondissement,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
5,Sunny flat 2 minutes from Eiffel Tower - null ...,http://airbnb.com/rooms/32696994?adults=1&prev...,(157),pas d'infos sur le prix,Logement entier à Tour Eiffel - Champ de Mars,4.41,2 voyageurs · Studio · 1 lit · 1 salle de bain
6,Grand F1 au cœur de Paris / Montorgueil - null...,http://airbnb.com/rooms/4962119?adults=1&previ...,(170),pas d'infos sur le prix,Logement entier à 2e arrondissement,4.88,4 voyageurs · Studio · 2 lits · 1 salle de bain
7,Chambre bleue - null - Paris,http://airbnb.com/rooms/45065696?adults=1&prev...,(25),pas d'infos sur le prix,Logement entier à Paris,4.44,"2 voyageurs · 1 chambre · 0 lit · 1,5 salle de..."
8,Studio for Student or Pro in Montmartre/St Laz...,http://airbnb.com/rooms/36326691?adults=1&prev...,(4),pas d'infos sur le prix,Logement entier à 9e arrondissement,4.75,1 voyageur · Studio · 1 lit · 1 salle de bain
9,Parisian Apartment - null - Paris,http://airbnb.com/rooms/36683089?adults=1&prev...,(14),pas d'infos sur le prix,Logement entier à Nation,4.64,2 voyageurs · Studio · 1 lit · 1 salle de bain
10,Studio charmant proche du marais - null - Paris,http://airbnb.com/rooms/31277703?adults=1&prev...,(9),pas d'infos sur le prix,Logement entier à 11e arrondissement,4.56,2 voyageurs · Studio · 1 lit · 1 salle de bain


In [37]:
url_page

'https://www.airbnb.fr/s/Paris/homes?adults=1&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&checkin=2020-11-25&checkout=2020-11-28&source=structured_search_input_header&search_type=filter_change'

In [38]:
## il faut ajouter des dates aux liens pour voir les prix
def findNextPage(soupPage):
	''' Finds the next page with listings if it exists '''
	try:
		nextpage = "https://airbnb.com" + soupPage.find("li", {"class": "_i66xk8d"}).find("a")["href"]
	except: # When he can't find the button, I assume he reached the end
		nextpage = "no next page"
	return nextpage
url=findNextPage(page)
url

'no next page'

In [39]:
## nombre de page du site web
def getPages(url):
	''' This function returns all the links to the pages containing 
	listings for one particular city '''
	result = []
	while url != "no next page": 
		page = getPage(url)
		result = result + [page]
		url = findNextPage(page)
	return result
URL=getPages(url_page)
len(URL)

1

In [40]:
## Scrapper toutes les infos sur les logements
def extractPages(url):
	''' This function outputs a dataframe that contains all information of a particular
	city. It thus contains information of multiple listings coming from multiple pages.'''
	pages = getPages(url)
	# Do for the first element to initialize the dataframe
	df =extractinfo(pages[0])
	# Loop over all other elements of the dataframe
	for pagenumber in range(1, len(pages)):
		df = df.append(extractinfo(pages[pagenumber]))
	return df
extractPages(url_page)

,title,link,nbComments,prix,topRow,evaluation,roomInfo
1,Cosy studio - Levallois/Champerret - null - Le...,http://airbnb.com/rooms/42826352?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 17e arrondissement,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
2,Appartement central - Louvre - null - Paris,http://airbnb.com/rooms/42825643?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 2e arrondissement,pas d'évaluation,4 voyageurs · 1 chambre · 2 lits · 1 salle de ...
3,MAISON LE BAC Apt. 402B - null - Paris,http://airbnb.com/rooms/45727652?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 10e arrondissement,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
4,Chambre romantique dans le 18ème arrondissemen...,http://airbnb.com/rooms/45212111?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Chambre privée à Montmartre,pas d'évaluation,"1 voyageur · 1 chambre · 1 lit · 1,5 salle de ..."
5,"Agréable 2 pièces non loin de Paris, idéal cou...",http://airbnb.com/rooms/46692343?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Appartement en résidence entier à 20e arrondis...,pas d'évaluation,2 voyageurs · 1 chambre · 1 salle de bain
6,Luxury Apartment Heart of Paris Bail Mobilité ...,http://airbnb.com/rooms/33893093?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à 1er arrondissement,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
7,Chambre saint germain - null - Paris,http://airbnb.com/rooms/44545731?adults=1&prev...,(34),pas d'infos sur le prix,Logement entier à Saint Germain des Prés - Odéon,4.24,2 voyageurs · Studio · 1 lit · 1 salle de bain
8,studio 19ème ( bail mobilité) - null - Paris,http://airbnb.com/rooms/45937788?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Logement entier à La Villette,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
9,Studio for Student or Pro in Montmartre/St Laz...,http://airbnb.com/rooms/36326691?adults=1&prev...,(4),pas d'infos sur le prix,Logement entier à 9e arrondissement,4.75,1 voyageur · Studio · 1 lit · 1 salle de bain
10,Studette Pour une personne unique - null - Paris,http://airbnb.com/rooms/42491068?adults=1&prev...,pas de commentaire,pas d'infos sur le prix,Suite entière à 18e arrondissement,pas d'évaluation,1 voyageur · Studio · 1 lit · 1 salle de bain


In [41]:
##base de tous les logements avec les évaluations
data=pd.DataFrame.from_dict(extractPages(url_page))

In [ ]:
## scrapper les commentaires de chaque logement

In [45]:
def setupDriver(url, waiting_time = 2.5):
	''' Initializes the driver of selenium'''
	driver = webdriver.Chrome()
	driver.get(url)
	time.sleep(waiting_time) 
	return driver

In [61]:
def getJSpage(url):
	''' Extracts the html of the webpage including the JS elements,
	output should be used as the input for all functions extracting specific information
	from the detailed pages of the listings '''
	driver = setupDriver(url)
	read_more_buttons = driver.find_elements_by_class_name("_1d079j1e")
	try: # not all pages have buttons
		for i in range(2, len(read_more_buttons)):
			read_more_buttons[i].click()
	except:
		pass
	html = driver.page_source
	driver.close()
	return BeautifulSoup(html, features="lxml") 

getJSpage(url_page)

<html class="js-focus-visible" data-is-hyperloop="true" data-triggered="true" dir="ltr" lang="fr" xmlns:fb="http://ogp.me/ns/fb#"><head><script async="" crossorigin="anonymous" src="https://connect.facebook.net/fr_FR/sdk.js?hash=3c369b98368fc327e638742b0f0cfa69&amp;ua=modern_es6"></script><script async="" crossorigin="anonymous" integrity="sha384-CQpJlyAOkK47dHQS0xIoX5twT2dgzOQPY4WUBqgn+uYNkQAIdu092kbVI2bFKtUe" src="https://www.gstatic.com/recaptcha/releases/UFwvoDBMjc8LiYc1DKXiAomK/recaptcha__fr.js" type="text/javascript"></script><script>window.sherlock_firstbyte = window.performance && window.performance.timing ? window.performance.timing.responseStart : Number(new Date());</script><script>!function(){"use strict";var n="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/",o=new RegExp("^\\d{10}_[".concat(n,"]{").concat(16,"}$"));var e=/(?:^| )bev=(.*?)(?:;|$)/,t=!1;function c(){if(t||"undefined"==typeof document)return null;t=!0;var n=(document.cookie||"").match(e);if(

In [62]:
def getAmenitiesPage(detailpage):
    '''récupère le html de la page web contenant les informations
     sur les équipements disponibles dans la pièce '''
    
    link = detailpage.find(class_ = "_1v4ygly5")["href"]
    driver = setupDriver("https://airbnb.com" + link, 5) # La page des équipements 
                                                        # est un lien déguisé en bouton, c'est pourquoi je dois faire ceci
    
    html = driver.page_source
    driver.close()
    return BeautifulSoup(html, features="lxml")

In [63]:
first = True # These variables were coded in a smarter way when doing the actual analysis
scraped = 0  # It used the length of the intermediate_results_par dataset stored on the pc

In [64]:
# exportation de données sur mon pc

def getAddis(url): 
    ''' This function is used to extract the html of the additional pages (detail page and amenities page)'''
    global first
    global scraped
    output = pd.DataFrame(columns=["details_page", "amenities_page", "link"])
    try:
        dp = getJSpage(url)
        output.loc[0] = [dp, getAmenitiesPage(dp), url]
    except:
        output.loc[0] = [-1, -1, url]
    if first: # Ensures that the columns have the correct titles because apparently that's difficult
        output.to_csv('intermediate_results_par.csv', mode='a', header=True, index = False)
        first = False
    else:
        output.to_csv('intermediate_results_par.csv', mode='a', header=False, index = False) 
    scraped += 1
    print("Scraped: {}".format(scraped))

In [65]:
# Extract Javascript enabled information    
def getReviews(detailpage):
    ''' Returns a list of the featured reviews on the page '''
    reviews = detailpage.findAll(class_ = "_50mnu4")
    output = ""
    for review in reviews:
        output += review.text + "**-**" #**-** can be used to split reviews later again
    return output


In [66]:
def getAmenities(amenitiespage):
    amenities = amenitiespage.findAll(class_ = "_vzrbjl")
    output = ""
    for amenity in amenities:
        output += re.findall('[A-Z][^A-Z]*', amenity.text)[0] + "**-**" # **-** will be used to split the string later for the purpose of dummification
    return output



In [67]:
def getResponseInfo(detailpage):
    try:
        output = detailpage.find(class_ = "_jofnfy").text
    except:
        output = ""
    return output

In [68]:
# Avec seulement beautifulsloup, on nettoie les fonctions pour les données de base extraites

def cleanFacilities(df): # Treating the facilities as a bag of words to create dummy variables
    df.loc[:, "facilities"] = df["facilities"].astype(str).str.replace("[","").str.replace("]","")
    vectorizer = CountVectorizer(decode_error = "ignore") 
    X = vectorizer.fit_transform(df.facilities)
    bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    return pd.concat([df.reset_index(drop=True).drop("facilities", axis = 1), bag_of_words], axis=1)



In [69]:
def cleanTitle(df):
    df.loc[:, "name"] = df["title"].str.split(" null ", n = 0, expand = True)[0].str.replace("-", "")
    df.loc[:, "location"] = df["title"].str.split(" null ", n = 0, expand = True)[1].str.replace("-", "").str.strip()
    return df.drop("title", axis = 1)




In [70]:
def cleanTopRow(df):
    df.loc[:, 'roomtype'] = df["toprow"].str.split(" in ", n = 0, expand = True)[0] 
    df.loc[:, 'detailed_location'] = df["toprow"].str.split(" in ", n = 0, expand = True)[1] 
    return df.drop("toprow", axis = 1)



In [71]:
def cleanRoomInfo(df):
    df.loc[:, "guests"] = df.loc[:, "roominfo"].str.split(" · ", n = 0, expand = True)[0].str.replace(" guests", "")
    df.loc[:, "bedrooms"] = df.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[1]
    df.loc[:, "beds"] = df.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[2].str.replace(" bed", "").str.replace("s", "")
    df.loc[:, "bathrooms"] = df.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[3]
    df.loc[:, "guests"] = pd.to_numeric(df.guests, errors = 'coerce')
    df.loc[:, "beds"] = pd.to_numeric(df.beds, errors = 'coerce')
    df.loc[:, "bedrooms"] = pd.to_numeric(df.bedrooms.str.split(" ", n = 0, expand = True)[0], errors = "ignore")
    df.loc[:, "bathrooms"] = pd.to_numeric(df.bathrooms.str.split(" ", n = 0, expand = True)[0], errors = "ignore")
    return df.drop("roominfo", axis = 1)

In [72]:
def cleanPrice(df):
    df.loc[:, "pricepernight"] = df.loc[:, "price"].str.split("Discounted", n = 0, expand = True)[0].str.replace("$", "/").str.split("/",  n = 0, expand = True)[1]
    df.loc[:, 'discountedpricepernight'] = df.loc[:, "price"].str.split("Discounted", n = 0, expand = True)[1].str.replace("$", "/").str.split("/",  n = 0, expand = True)[1]
    df.loc[:, "price"] = pd.to_numeric(df.pricepernight.str.replace(",","").str.strip())
    df.loc[:, "discountedprice"] = pd.to_numeric(df.discountedpricepernight.str.replace(" ", "").str.replace(",",""), errors = "coerce")
    return df.drop(["pricepernight", "discountedpricepernight"], axis = 1)



In [73]:
def cleanRating(df):
    df.loc[:, "score"] = df.loc[:, 'rating'].str.split(" ", n = 0, expand = True)[1]
    df.loc[:, "score"] = pd.to_numeric(df.score, errors = "coerce")
    return df.drop("rating", axis = 1)



In [74]:
def cleanReviewNumber(df):
    df.loc[:, "reviewnumber"] = df.loc[:, 'reviewnumber'].str.split(" ", n = 0, expand = True)[0]
    df.loc[:, "reviewnumber"] = pd.to_numeric(df.reviewnumber, errors = "coerce")
    return df



In [75]:
def clean(df):
    df = cleanTitle(df)
    df = cleanFacilities(df)
    df = cleanTopRow(df)
    df = cleanRoomInfo(df)
    df = cleanPrice(df)
    df = cleanRating(df)
    df = cleanReviewNumber(df)
    # Reorder columns
    col1 = df.pop('price')
    df = pd.concat([df.reset_index(drop=True), col1], axis=1)
    col2 = df.pop('reviewnumber')
    df = pd.concat([df.reset_index(drop=True), col2], axis=1) 
    col3 = df.pop('link')
    df = pd.concat([df.reset_index(drop=True), col3], axis=1) 
    return df

In [76]:
# fonctions pour nettoyer les données contenant les html des pages supplémentaires

def cleanAmenities(df):
    df.loc[:, "amenities"] = df.amenities.replace(np.nan, '', regex=True)# fit_transform ne peut pas traiter les valeurs manquantes
    
    df.loc[:, "amenities"] = df.amenities.str.replace(" ", "_").str.replace("-", " ").str.replace("*", "") #sectionné en deux à cause d'un problème de bug sur python
                                                                                                            #(https://stackoverflow.com/questions/3675144/regex-error-nothing-to-repeat)
    vectorizer = CountVectorizer(decode_error = "ignore") 
    X = vectorizer.fit_transform(df.amenities)
    bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    return pd.concat([df.reset_index(drop=True).drop("amenities", axis = 1), bag_of_words], axis=1)


In [77]:
def cleanReviews(df):
    df.loc[:, "reviews"] = df.reviews.replace(np.nan, '', regex=True)# fit_transform cannot handle missing values
    df.loc[:, "reviews"] = df.reviews.str.split("-")
    return df

In [78]:
def getResponseTime(string):
    if "Response time" in string:
        output = string[string.find("Response time") + 15:]
    else:
        output = "Unknown"
    return output


In [79]:
def getResponseRate(string):
    if "Response rate" in string:
        temp = string[string.find("Response rate") + 15:string.find("Response rate")+20] 
        output = ""
        for letter in temp:
            if letter in "0123456789":
                output += letter
    else:
        output = "Unknown"      
    return output


In [80]:
def getLanguages(string):
    if "Language" in string:
        if "Response" in string:
            output = string[10:string.find("Response")].strip()
        else:
            output = string[10:].strip()
    else:
        output = "Unknown"
    return output

In [81]:
def cleanResponseTime(df):
    df.loc[:, "response_info"] = df.response_info.replace(np.nan, '', regex=True)
    df.loc[:, "response_time"] = df.response_info.apply(lambda x: getResponseTime(x))
    return df

In [82]:
def cleanResponseRate(df):
    df.loc[:, "response_rate"] = df.response_info.apply(lambda x: getResponseRate(x))
    return df

In [83]:
def cleanLanguages(df):
    df.loc[:, "languages"] = df.response_info.apply(lambda x: getLanguages(x))
    df.loc[:, "languages"] = df.languages.str.split(",")
    return df

In [84]:
def cleanResponseInfo(df):
    df = cleanResponseTime(df)
    df = cleanResponseRate(df)
    df = cleanLanguages(df)
    return df.drop("response_info", axis = 1)

In [90]:
# scraper

def scraper(urls, sample_size = None, random_state = 1234):
    df = extractPages(url)
    df = clean(df)
    if sample_size is not None:
        df = df.sample(sample_size, random_state = random_state)
    Parallel(n_jobs = -1, prefer="threads")(delayed(getAddis)(url) for url in df.link)
    df2 = pd.read_csv("intermediate_results_par.csv")
    df = df.merge(df2, on = "link")
    df.loc[:, 'reviews'] = df.details_page.apply(lambda x: getReviews(BeautifulSoup(x, features = "lxml")))
    df.loc[:, 'response_info'] = df.details_page.apply(lambda x: getResponseInfo(BeautifulSoup(x, features = "lxml")))
    df.loc[:, "amenities"] = df.amenities_page.apply(lambda x: getAmenities(BeautifulSoup(x, features = "lxml")))
    df = cleanReviews(df)
    df = cleanResponseInfo(df)
    df = cleanAmenities(df)
    return df

In [91]:
df = scraper(url_page, sample_size = 10)

IndexError: list index out of range